# Fine-Tune Whisper For Werewolf

<img src="https://raw.githubusercontent.com/sanchit-gandhi/notebooks/main/whisper_architecture.svg" alt="Trulli" style="width:100%">

## Prepare Environment

In [ ]:
!pip install --quiet datasets evaluate jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.2 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## Consts

In [ ]:
BATCH_SIZE = 50
BOS_LEN = 2
EOS_LEN = 1
MAX_DURATION = 30
MASK_ID = -100
MAX_LENGTH = 448
SAMPLING_RATE = 16000
WORD_ERROR_PENALTY = 100

## Load Model

In [ ]:
from transformers import WhisperFeatureExtractor, WhisperTokenizer, WhisperProcessor, WhisperForConditionalGeneration

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small")
processor = WhisperProcessor.from_pretrained("openai/whisper-small")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

## Load Dataset

In [ ]:
from datasets import load_dataset, Audio, DatasetDict, Dataset

werewolf_data = DatasetDict()

werewolf_data["train"] = load_dataset("parquet", data_files=["https://huggingface.co/datasets/iohadrubin/werewolf_dialogue_data_10sec/resolve/main/data/train-00002-of-00014-e0e6b0000eedceb4.parquet"])["train"]
werewolf_data["test"] = load_dataset("parquet", data_files=["https://huggingface.co/datasets/iohadrubin/werewolf_dialogue_data_10sec/resolve/main/data/train-00009-of-00014-603088eeb6352a9b.parquet"])["train"]

werewolf_data = werewolf_data.filter(lambda x: x["end"] - x["start"] <= MAX_DURATION and x["dialogue"][-1]["target"] is not None)
werewolf_data = werewolf_data.remove_columns(["start", "end", "idx", "Game_ID", "file_name", "video_name", "startRoles", "startTime", "endRoles", "playerNames"])

(…)-00002-of-00014-e0e6b0000eedceb4.parquet:   0%|          | 0.00/402M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

(…)-00009-of-00014-603088eeb6352a9b.parquet:   0%|          | 0.00/427M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Filter:   0%|          | 0/911 [00:00<?, ? examples/s]

Filter:   0%|          | 0/910 [00:00<?, ? examples/s]

## Prepare Dataset

In [ ]:
prompt_prefix, prompt_suffix = """Given the following dialogue and audio, assign the last utterance one or more of the following tags (delimited by commas):
'Accusation', 'Defense', 'Evidence', 'Identity Declaration', 'Interrogation', 'No Strategy'

```
""", """
```

Reminder - Assign one or more of the following tags to the last utterance (delimited by commas):
'Accusation', 'Defense', 'Evidence', 'Identity Declaration', 'Interrogation', 'No Strategy'

Assignment:
"""

prompt_prefix_len = len(tokenizer.encode(prompt_prefix)) - EOS_LEN
max_prompt_len = MAX_LENGTH - prompt_prefix_len

In [ ]:
import numpy as np

def prepare_decoder_input_ids_and_labels(sample):
  dialogue = "\n".join(f"{x['speaker']}: {x['utterance']}" for x in sample["dialogue"])
  target = sample["dialogue"][-1]["target"]
  text = prompt_prefix + dialogue + prompt_suffix + target

  input_ids = tokenizer.encode(text)
  if len(input_ids) > MAX_LENGTH:
    input_ids = input_ids[:prompt_prefix_len] + input_ids[-max_prompt_len:]

  decoder_input_ids = np.array(input_ids)
  sample["decoder_input_ids"] = decoder_input_ids

  labels = np.array(input_ids)
  target_len = len(tokenizer.encode(target)) - BOS_LEN
  labels[:-target_len] = MASK_ID
  sample["labels"] = labels

  return sample

def prepare_audio(batch):
  audio_arrays = [x["array"] for x in batch["audio"]]

  batch["input_features"] = feature_extractor(audio_arrays, sampling_rate=SAMPLING_RATE, return_tensors="pt").input_features

  return batch

In [ ]:
werewolf_data = werewolf_data.map(prepare_decoder_input_ids_and_labels)
werewolf_data = werewolf_data.map(prepare_audio, batched=True, batch_size=BATCH_SIZE)

Map:   0%|          | 0/901 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1053 > 1024). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/896 [00:00<?, ? examples/s]

Map:   0%|          | 0/901 [00:00<?, ? examples/s]

Map:   0%|          | 0/896 [00:00<?, ? examples/s]

## Data Collator

In [ ]:
from dataclasses import dataclass
from transformers import WhisperProcessor

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: WhisperProcessor
    decoder_start_token_id: int

    def __call__(self, features):
      batch = self.processor.feature_extractor.pad([{"input_features": feature["input_features"]} for feature in features], return_tensors="pt")

      decoder_input_ids_batch = self.processor.tokenizer.pad([{"input_ids": feature["decoder_input_ids"]} for feature in features], return_tensors="pt")
      batch["decoder_input_ids"] = decoder_input_ids_batch["input_ids"]

      labels_batch = self.processor.tokenizer.pad([{"input_ids": feature["labels"]} for feature in features], return_tensors="pt")
      labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), MASK_ID)
      batch["labels"] = labels

      return batch

In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

## Compute Metrics

In [ ]:
import evaluate

metric = evaluate.load("wer")

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    label_ids[label_ids == MASK_ID] = tokenizer.pad_token_id

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = WORD_ERROR_PENALTY * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

## Training Arguments

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-werewolf",
    eval_strategy="steps",
    eval_steps=1000,
    max_steps=4000,
    warmup_steps=500,
    logging_steps=25,
    save_steps=1000,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    report_to=["tensorboard"],
    push_to_hub=True,
    gradient_checkpointing=True,
    predict_with_generate=True
)

## Trainer

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=werewolf_data["train"],
    eval_dataset=werewolf_data["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

### Training

In [ ]:
trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Step,Training Loss,Validation Loss


Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


ValueError: one or more references are empty strings

## Publish

In [ ]:
kwargs = {
    "dataset_tags": "iohadrubin/werewolf_dialogue_data_10sec",
    "dataset_args": "split: test",
    "model_name": "Whisper Small Werewolf",
    "finetuned_from": "openai/whisper-small",
    "tasks": "classification",
}

In [ ]:
trainer.push_to_hub(**kwargs)